# Gerrymandering and the Redistricting Problem

## Assumptions

We assume a 2-party system with the parties represented as "D" and "R". We start by only using one period's worth of historical data. 

## Data Inputs

The expected number of votes for each party is a key data input for our constraints. We represent this with a matrix, $\mathbf{V}$. Since we have two parties, the matrix has the following shape.

$$ \mathbf{V} \in \mathbb{R}^{|blocks| \times 2} $$

$blocks$ is a set where $|blocks|$ is the number of elements in the set. By convention, the first column of $\mathbf{V}$ will be D votes and the second column R votes. $\mathbf{V}$ could represent a single past election's results, or it could be an expected upcoming result based on an exogenous model.

## Variables

We represent the variables with a matrix. Each row of the matrix represents an indivisible block (precint, county, or census area depending on the conventions of the problem). The columns represent assignment to a district. The matrix is made up of zeroes or ones, and each row must have exactly one entry equal to one, meaning that each row must be in one and only one district. $districts$ is a set where $|districts|$ is the number of elements in each set. 

$$ 
\mathbf{D} \in \{0,1\}^{|blocks| \times |districts|}. 
$$

For ease of interpretation, we're going to define another variable that represents the "efficiency gap" in votes. To do this, we need to know the number of expected votes for each party in each block and the assignment of blocks to districts. $\mathbf{D}^T \mathbf{V} \in \mathbb{R}^{|districts| \times 2} $ gives us a matrix of the number of D and R votes in each district. The efficiency gap can be calculated from the seat margin and vote margin. [Brennan Center](https://www.brennancenter.org/sites/default/files/legal-work/How_the_Efficiency_Gap_Standard_Works.pdf)

$$ \text{Efficiency Gap} = (\text{Seat Margin} - 50\%) - 2 ( \text{Vote Margin} - 50\% ) $$

The vote margin is easily calculated by multiplying $\mathbf{D^TV}$ by a vector of ones, $\mathbf{j}$, and subtracting the elements. The seat margin is trickier, since we need to determine which of the two parties is the winner for any given potential solution.  We do this by introducing another vector of variables $\mathbf{w}\in \{0,1\}^{|districts|}$, indicating if D won the given seat (or district). 

$$ \mathbf{w}\in \{0,1\}^{|districts|} $$

When we include $w$ in our objective with an appropriately large penalty, the optimization problem will set $w_i = 1$ if D won the seat and $0$ otherwise. Our constraints will ensure this. So the margin variables are now: 

$$ \text{Seat Margin} = \frac{ (\mathbf{w}\cdot \mathbf{j} - (1-\mathbf{w})\cdot \mathbf{j} )}{|districts| } $$

$$ \text{Vote Margin} = \frac{\mathbf{j^TD^TV} \left[ \begin{array}{c} 1 \\ -1 \end{array} \right]}{\mathbf{j^TVj}} $$

In a given district's race, the number of wasted votes can be calculated szz


## Constraints

To force the $\mathbf{w}$ variable, set a constraint (vector valued):
$$ \mathbf{D^TV} \left[ \begin{array}{c} 1 \\ -1 \end{array} \right] \geq \mathbf{w} $$. 




## Example Problem

In [90]:
V = [75 25; 60 40; 43 57; 48 52; 49 51]


5×2 Array{Int64,2}:
 75  25
 60  40
 43  57
 48  52
 49  51

In [91]:
using JuMP 
using AmplNLWriter
using CoinOptServices
using ECOS
using GLPKMathProgInterface
using NLopt

In [100]:
blocks = size(V,1)
districts = 5
t = ones(districts,1)
total_vote = V * ones(2,1)
votes_to_win = ceil.(total_vote / 2 + 0.1)

#m = Model(solver = AmplNLSolver(CoinOptServices.bonmin, [
#    "bonmin.pump_for_minlp=yes"]) )
#m = Model(solver = ECOSSolver() )
m = Model(solver = GLPKSolverMIP())
# m = Model(solver=NLoptSolver(algorithm=:LD_SLSQP))

## Variables 

@variable(m, 0 <= D[i=1:blocks,j=1:districts] <= 1 , Bin)
@variable(m, 0 <= w[i=1:districts] <= 1, Bin)
@variable(m, min_vote[i=1:districts])
@variable(m, eff_gap)
@variable(m, abs_eff_gap)
@variable(m, wasted_u[i=1:districts, j=1:2])
@variable(m, wasted_o[i=1:districts, j=1:2])


## Constraints

@constraint(m, D .== eye(districts))

@constraint(m, D * ones(districts,1) .== 1)  # each block can be in only one district

# These constraints set w to 1 if D wins in a district
M = districts * sum(total_vote) * 1000
@constraint(m, min_vote .>= (D' * V)[:,1] - M * (w) )
@constraint(m, min_vote .>= (D' * V)[:,2] - M * (1-w) )

# These constraints set wasted_u to the number of wasted votes for the losing parties
@constraint(m, wasted_u .>= 0)
@constraint(m, wasted_u[:,1] .>= (D' * V)[:,1] - M * w)
@constraint(m, wasted_u[:,2] .>= (D' * V)[:,2] - M * (1-w))

# These constaints set wasted_o to the number of wasted votes for the winning parties
@constraint(m, wasted_o .>= 0)
@constraint(m, wasted_o .>= (D' * V) - votes_to_win)

# These constraints calculate the efficiency gap
@constraint(m, eff_gap .== ones(2,districts) * (wasted_u + wasted_o) * [1;-1])
@constraint(m, abs_eff_gap >= eff_gap)
@constraint(m, abs_eff_gap >= - eff_gap)


## Objective

@objective(m, Min, sum(min_vote) + sum(wasted_u) + sum(wasted_o) ) # abs_eff_gap + sum(w) )

#m

min_vote[1] + min_vote[2] + min_vote[3] + min_vote[4] + min_vote[5] + wasted_u[1,1] + wasted_u[2,1] + wasted_u[3,1] + wasted_u[4,1] + wasted_u[5,1] + wasted_u[1,2] + wasted_u[2,2] + wasted_u[3,2] + wasted_u[4,2] + wasted_u[5,2] + wasted_o[1,1] + wasted_o[2,1] + wasted_o[3,1] + wasted_o[4,1] + wasted_o[5,1] + wasted_o[1,2] + wasted_o[2,2] + wasted_o[3,2] + wasted_o[4,2] + wasted_o[5,2]

In [93]:
@time begin
    status = solve(m)
end

  0.014151 seconds (883 allocations: 84.047 KiB)


:Optimal

In [94]:
println("objective: ", getobjectivevalue(m))


_D = getvalue(D)
_D' * V
getvalue(y)
getvalue(w)

getvalue(wasted_o)

getvalue(abs_eff_gap)
getvalue(wasted_u)
getvalue(wasted_o)
getvalue(eff_gap)

objective: 450.0


101.0

In [95]:
getvalue(wasted_u)

5×2 Array{Float64,2}:
 -0.0  25.0
 -0.0  40.0
 43.0  -0.0
 48.0  -0.0
 49.0  -0.0

In [96]:
getvalue(wasted_o)

5×2 Array{Float64,2}:
 24.0  -0.0
  9.0  -0.0
 -0.0   6.0
 -0.0   1.0
 -0.0  -0.0

In [97]:
getvalue(eff_gap)

101.0

In [101]:
V = [75 25; 60 40; 43 57; 48 52; 49 51]

D = eye(5)
(D' * V) 
ceil.(V * ones(2,1) / 2 + 0.1)
V
D' * V

5×2 Array{Float64,2}:
 75.0  25.0
 60.0  40.0
 43.0  57.0
 48.0  52.0
 49.0  51.0

In [20]:
V[:,1] .* [1;2;3;4;5]

5-element Array{Int64,1}:
  75
 120
 129
 192
 245